In [6]:
pip install country_converter

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: Could not find a suitable TLS CA certificate bundle, invalid path: C:\Users\fabia\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\certifi\cacert.pem



In [7]:
# -*- coding: utf-8 -*-
# NORMALIZAR NOMBRES DE PAÍS Y ASIGNAR ISO3 (SIN USAR pycountry o country_converter)

import pandas as pd
import unicodedata
import re
from pathlib import Path

# --- 1) CONFIGURACIÓN DE RUTAS ---
CONCAT_PATH = "../data/raw/combined_dataset.csv"  # ← Ajusta aquí si tu archivo tiene otro nombre
OUT_DIR = Path("../data/processed_csv")
OUT_DIR.mkdir(exist_ok=True, parents=True)
OUT_PATH = OUT_DIR / "processed_csv_concat_with_iso3.csv"
UNMAPPED_PATH = OUT_DIR / "unmapped_countries.csv"

# --- 2) HELPERS DE LIMPIEZA ---
def strip_accents(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s))
    return "".join(ch for ch in s if not unicodedata.combining(ch))

def clean_name(s: str) -> str:
    s = strip_accents(s).lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^a-zA-Z\s]", "", s)  # quitar símbolos
    return s

# --- 3) LEER EL DATASET CONCATENADO ---
df = pd.read_csv(CONCAT_PATH, low_memory=False)
print("Filas cargadas:", len(df))

# Detectar nombre de columna país si no se llama country
country_col = None
for c in df.columns:
    if "country" in c.lower() or "pais" in c.lower():
        country_col = c
        break

if country_col is None:
    raise ValueError("No se encontró columna de país. Renombra o ajusta manualmente.")

print("Columna de país detectada:", country_col)

# Crear versión normalizada
df["country_clean"] = df[country_col].astype(str).apply(clean_name)

# --- 4) DICCIONARIO CURADO LATAM → ISO3 ---
manual_iso_map = {
    "colombia": "COL",
    "argentina": "ARG",
    "bolivia plurinational state of": "BOL",
    "bolivia": "BOL",
    "brazil": "BRA",
    "brasil": "BRA",
    "chile republic of": "CHL",
    "chile": "CHL",
    "costa rica": "CRI",
    "ecuador": "ECU",
    "el salvador": "SLV",
    "guatemala": "GTM",
    "honduras": "HND",
    "mexico": "MEX",
    "nicaragua": "NIC",
    "panama": "PAN",
    "paraguay": "PRY",
    "peru": "PER",
    "uruguay": "URY",
    "venezuela": "VEN",
    "haiti": "HTI",
    "dominican republic": "DOM",
    "republica dominicana": "DOM"
}

# --- 5) APLICAR MAPEOS ---
df["iso3"] = df["country_clean"].map(manual_iso_map)

# --- 6) REVISAR NO MAPEADOS ---
unmapped = df[df["iso3"].isna()][country_col].unique()
print("\nPaíses no mapeados:", unmapped)

pd.Series(unmapped).to_csv(UNMAPPED_PATH, index=False, header=["country_unmapped"])
print("Archivo de países no mapeados guardado en:", UNMAPPED_PATH)

# --- 7) GUARDAR RESULTADO FINAL ---
df.to_csv(OUT_PATH, index=False)
print("\nArchivo final guardado en:", OUT_PATH)

# --- 8) RESUMEN ---
total = len(df)
missing = df["iso3"].isna().sum()
print(f"\nTotal filas: {total}  - Filas sin iso3: {missing}  ({missing/total:.1%})")


Filas cargadas: 95
Columna de país detectada: Country

Países no mapeados: []
Archivo de países no mapeados guardado en: ..\data\processed_csv\unmapped_countries.csv

Archivo final guardado en: ..\data\processed_csv\processed_csv_concat_with_iso3.csv

Total filas: 95  - Filas sin iso3: 0  (0.0%)
